# Superhero data analysis

Shenyue Jia

In [77]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import json

## Get data

In [78]:
# df_info as superhero info data
df_info = pd.read_csv('Data/superhero_info.csv')
df_info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [79]:
# df_powers as superhero info data
df_powers = pd.read_csv('Data/superhero_powers.csv')
df_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


## Clean and combine data

- Separate `Hero|Publisher` column by `|`
- Decompose dictionary in `Measurements` to `Height` and `Weight`
- Convert `Powers` column into one-hot-encoder columns 
    - `Agility`
    - `Flight`
    - `Superspeed`
    - The rest of possible `Powers` types

### Separate `Hero|Publisher` column

In [80]:
## save the 2 new columns into the dataframe
df_info[['Hero','Publisher']] = df_info['Hero|Publisher'].str.split('|',expand=True)
df_info = df_info.drop(columns = ['Hero|Publisher'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


### Decomposing `Measurements` column

- First, convert the `Measurements` column into actual dictionaries

In [81]:
# Analyze one record of Measurements column
test = df_info.loc[0,"Measurements"]
type(test)

str

In [82]:
test

"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

- The string of `test` contains double quotes, which need to be replaced so that it can be converted to a dictionary

In [83]:
test = test.replace("'",'"')
test

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [84]:
# Apply the change to the entire column
## use .str.replace to replace all single quotes
df_info['Measurements'] = df_info['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
df_info['Measurements'] = df_info['Measurements'].apply(json.loads)
df_info['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

- Second, unpack the dictionaries into separate columns

In [85]:
# convert Measurements into a dataframe with two columns
hw = df_info['Measurements'].apply(pd.Series)
hw

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


- Concatenate two new columns to the `df_info` dataframe and drop the `Measurements` column

In [86]:
# concat hw with original dataframe
df_info = pd.concat((df_info, hw), axis = 1)
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [87]:
# drop Measurements column
df_info = df_info.drop(columns=['Measurements'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


### Unpack the `Powers` column in `df_powers`

In [88]:
test = df_powers.loc[0,'Powers']
test

'Agility,Super Strength,Stamina,Super Speed'

In [89]:
type(test)

str

- First, in order to use the `.explode` method, we need to convert `str` type to `list` type

In [90]:
def convert_to_list(string):
    li = list(string.split(","))
    return li

In [91]:
test = convert_to_list(test)
test

['Agility', 'Super Strength', 'Stamina', 'Super Speed']

In [94]:
# apply to the entire column
df_powers['Powers'] = df_powers['Powers'].apply(lambda x: list(x.split(",")))
df_powers['Powers'].head()

0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: Powers, dtype: object

In [95]:
# Check if the type is correct
type(df_powers.loc[0,'Powers'])

list

- Next, obtain unique values for `Powers` with `.explode` method

In [97]:
## exploding the column of lists
exploded = df_powers.explode('Powers')
exploded[['hero_names','Powers']].head(5)

,hero_names,Powers
0,3-D Man,Agility
0,3-D Man,Super Strength
0,3-D Man,Stamina
0,3-D Man,Super Speed
1,A-Bomb,Accelerated Healing


- Obtain unique names of columns to create for one-hot-encoder columns

In [98]:
## saving the unique values from the exploded column
cols_to_make = exploded['Powers'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

- Finally, use a for loop and `.str.contains` to create new columns

In [101]:
for col in cols_to_make:
    df_powers[col] = df_powers['Powers'].str.contains(col,na=False)
df_powers.head()

,hero_names,Powers,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,[Lantern Power Ring],False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre...",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [102]:
# prepare a dataframe to be combined with df_info
df_powers_clean = df_powers.drop(columns=['Powers'])
df_powers_clean.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Concatenate `df_info` and `df_powers` dataframes

- We can use `Hero` column in `df_info` and `hero_names` in `df_powers` as keys to join the two dataframes

In [144]:
## merging with different column names
merged = pd.merge(df_info, df_powers, left_on='Hero', right_on='hero_names')
merged.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg,...,False,False,False,False,False,False,False,False,False,False


## Explanatory Data Analysis

### 1. Compare the average weight of super powers who have Super Speed to those who do not.

In [129]:
list(merged.columns.values)

['Gender',
 'Race',
 'Alignment',
 'Hair color',
 'Eye color',
 'Skin color',
 'Hero',
 'Publisher',
 'Height',
 'Weight',
 'hero_names',
 'Powers',
 'Agility',
 'Super Strength',
 'Stamina',
 'Super Speed',
 'Accelerated Healing',
 'Durability',
 'Longevity',
 'Camouflage',
 'Self-Sustenance',
 'Cold Resistance',
 'Underwater breathing',
 'Marksmanship',
 'Weapons Master',
 'Intelligence',
 'Telepathy',
 'Immortality',
 'Reflexes',
 'Enhanced Sight',
 'Sub-Mariner',
 'Lantern Power Ring',
 'Invulnerability',
 'Animation',
 'Super Breath',
 'Dimensional Awareness',
 'Flight',
 'Size Changing',
 'Teleportation',
 'Magic',
 'Dimensional Travel',
 'Molecular Manipulation',
 'Energy Manipulation',
 'Power Cosmic',
 'Energy Absorption',
 'Elemental Transmogrification',
 'Fire Resistance',
 'Natural Armor',
 'Heat Resistance',
 'Matter Absorption',
 'Regeneration',
 'Stealth',
 'Power Suit',
 'Energy Blasts',
 'Energy Beams',
 'Heat Generation',
 'Danger Sense',
 'Phasing',
 'Force Fields',


In [154]:
merged_cp = merged.loc[:,['Hero','Weight','Super Speed']]
# merged_cp['Super Speed'] = merged_cp['Super Speed'].map({True: 'yes', False: 'no'})
# merged_cp.head()

In [155]:
avg_weight = merged_cp.groupby('Super Speed')['Weight'].mean().reset_index()
avg_weight

TypeError: Could not convert 441.0 kg65.0 kg90.0 kg441.0 kg122.0 kg88.0 kg81.0 kg104.0 kg108.0 kg90.0 kg90.0 kg169.0 kg173.0 kg101.0 kg68.0 kg57.0 kg54.0 kg83.0 kg90.0 kg122.0 kg86.0 kg358.0 kg135.0 kg106.0 kg146.0 kg63.0 kg68.0 kg57.0 kg98.0 kg270.0 kg59.0 kg50.0 kg101.0 kg126.0 kg68.0 kg54.0 kg81.0 kg63.0 kg67.0 kg180.0 kg77.0 kg54.0 kg57.0 kg52.0 kg61.0 kg95.0 kg77.0 kg79.0 kg133.0 kg63.0 kg181.0 kg68.0 kg216.0 kg135.0 kg71.0 kg54.0 kg124.0 kg155.0 kg113.0 kg95.0 kg58.0 kg59.0 kg54.0 kg86.0 kg90.0 kg52.0 kg92.0 kg90.0 kg59.0 kg104.0 kg86.0 kg88.0 kg97.0 kg68.0 kg56.0 kg77.0 kg230.0 kg495.0 kg86.0 kg55.0 kg97.0 kg135.0 kg61.0 kg99.0 kg52.0 kg90.0 kg59.0 kg158.0 kg74.0 kg81.0 kg108.0 kg90.0 kg116.0 kg74.0 kg101.0 kg74.0 kg86.0 kg61.0 kg61.0 kg97.0 kg63.0 kg81.0 kg50.0 kg55.0 kg54.0 kg86.0 kg170.0 kg70.0 kg225.0 kg67.0 kg99.0 kg104.0 kg50.0 kg173.0 kg88.0 kg52.0 kg90.0 kg81.0 kg817.0 kg56.0 kg135.0 kg27.0 kg52.0 kg90.0 kg95.0 kg91.0 kg178.0 kg101.0 kg95.0 kg383.0 kg90.0 kg171.0 kg187.0

- In order to perform the calculation, we need to convert the `Weight` and `Height` columns into numeric values

In [156]:
test_w = merged.loc[0,'Weight']
test_w

'441.0 kg'

In [157]:
# split using space to get the number
test_w = float(test_w.split(' ')[0])
test_w

441.0

In [158]:
# apply to the entire 
merged_cp['Weight'] = merged_cp['Weight'].str.split(' ').str[0]
merged_cp['Weight'].head()

0    441.0
1     65.0
2     90.0
3    441.0
4    122.0
Name: Weight, dtype: object

In [159]:
# convert to numeric
merged_cp['Weight'] = pd.to_numeric(merged_cp['Weight'])

In [160]:
avg_weight = merged_cp.groupby('Super Speed')['Weight'].mean().reset_index()
avg_weight

,Super Speed,Weight
0,False,113.589633
